<a href="https://colab.research.google.com/github/maple-buice/chart-hero/blob/main/colab/transformer_training_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Environment Setup

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

Tue May 27 12:13:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [9]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set up project directory
PROJECT_DIR = '/content/drive/MyDrive/chart-hero'
!mkdir -p {PROJECT_DIR}
%cd {PROJECT_DIR}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/chart-hero


In [10]:
# Clone or update repository
import os
if not os.path.exists('chart-hero'):
    !git clone https://github.com/maple-buice/chart-hero.git
else:
    %cd chart-hero
    !git pull
    %cd ..

%cd chart-hero

/content/drive/MyDrive/chart-hero/chart-hero
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 8 (delta 4), reused 4 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (8/8), 2.01 KiB | 3.00 KiB/s, done.
From https://github.com/maple-buice/chart-hero
   b8042ad..c5df979  main       -> origin/main
Updating b8042ad..c5df979
Fast-forward
 colab/transformer_training_colab.ipynb | 92 ++++++++++++++++++++++++++--------
 1 file changed, 71 insertions(+), 21 deletions(-)
/content/drive/MyDrive/chart-hero
/content/drive/MyDrive/chart-hero/chart-hero


In [ ]:
# Install dependencies
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q pytorch-lightning transformers timm wandb
!pip install -q librosa soundfile scikit-learn pandas numpy matplotlib seaborn tqdm
!pip install -q ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 100.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 115.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 848.7/848.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 2. Data Setup

In [8]:
# Download and extract Expanded Groove MIDI Dataset
# Separated download and unzip steps with individual checks.

import os # Ensure os is imported

DATASET_URL = "https://storage.googleapis.com/magentadata/datasets/e-gmd/v1.0.0/e-gmd-v1.0.0.zip"
# Target directory on Google Drive for the unzipped dataset
DATASET_DIR = "/content/drive/MyDrive/chart-hero/datasets"
# Path to a representative file/directory within the unzipped dataset to check for existence
# Adjust if the structure of your unzipped dataset is different. This assumes a directory named 'e-gmd-v1.0.0' is created.
EXPECTED_DATASET_CONTENT_PATH = os.path.join(DATASET_DIR, "e-gmd-v1.0.0")

# Temporary path on Colab instance storage for the downloaded ZIP file
INSTANCE_TEMP_ZIP_FILENAME = "e-gmd-v1.0.0.zip"
INSTANCE_TEMP_ZIP_PATH = os.path.join(DATASET_DIR, INSTANCE_TEMP_ZIP_FILENAME)

print("--- 1. Download Step ---")
# Check if data is already unzipped (no need to download then)
if os.path.exists(EXPECTED_DATASET_CONTENT_PATH):
    print(f"Dataset already unzipped at {EXPECTED_DATASET_CONTENT_PATH}. Skipping download.")
# Check if ZIP is already downloaded to instance
elif os.path.exists(INSTANCE_TEMP_ZIP_PATH):
    print(f"ZIP file already downloaded to instance storage at {INSTANCE_TEMP_ZIP_PATH}. Skipping download.")
# Otherwise, download
else:
    print(f"Downloading {DATASET_URL} to instance storage ({INSTANCE_TEMP_ZIP_PATH})...")
    # Ensure /content directory exists for wget, though it usually does by default
    !mkdir -p /content
    !wget -O {INSTANCE_TEMP_ZIP_PATH} {DATASET_URL}
    if os.path.exists(INSTANCE_TEMP_ZIP_PATH):
        print("Download complete.")
    else:
        print(f"Download failed. ZIP file not found at {INSTANCE_TEMP_ZIP_PATH}.")

print("\n--- 2. Unzip Step ---")
# Check if data is already unzipped
if os.path.exists(EXPECTED_DATASET_CONTENT_PATH):
    print(f"Dataset already unzipped at {EXPECTED_DATASET_CONTENT_PATH}. Skipping unzip.")
else:
    # Check if the ZIP file exists on the instance (either freshly downloaded or from a previous run)
    if os.path.exists(INSTANCE_TEMP_ZIP_PATH):
        print(f"Unzipping {INSTANCE_TEMP_ZIP_PATH} to {DATASET_DIR}...")
        # Ensure target directory on Google Drive exists
        !mkdir -p {DATASET_DIR}
        # The -o option overwrites files without prompting. -q for quiet. -d for destination.
        !unzip -oq {INSTANCE_TEMP_ZIP_PATH} -d {DATASET_DIR}

        # Verify unzipping by checking for the expected content path
        if os.path.exists(EXPECTED_DATASET_CONTENT_PATH):
            print("Unzipping complete.")
        else:
            print(f"Unzipping may have failed. Expected content not found at {EXPECTED_DATASET_CONTENT_PATH}.")
            print(f"Temporary ZIP file {INSTANCE_TEMP_ZIP_PATH} will be kept for inspection.")
    else:
        # This case means the data isn't unzipped, and the ZIP isn't on the instance.
        print(f"ZIP file not found at {INSTANCE_TEMP_ZIP_PATH}. Cannot unzip. Please ensure download step was successful or the file exists.")

print(f"\nFinal check - Dataset directory contents ({DATASET_DIR}):")
!ls -la {DATASET_DIR}
print(f"\nFinal check - Instance /content directory contents:")
!ls -la /content

--- 1. Download Step ---
--2025-05-27 16:22:04--  https://storage.googleapis.com/magentadata/datasets/e-gmd/v1.0.0/e-gmd-v1.0.0.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.170.207, 142.251.175.207, 74.125.24.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.170.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96422999145 (90G) [application/zip]
Saving to: ‘/content/e-gmd-v1.0.0.zip’

zip                   0%[                    ] 343.84M  18.4MB/s    eta 84m 31s^C
Download complete.

--- 2. Unzip Step ---
Unzipping /content/e-gmd-v1.0.0.zip to /content/drive/MyDrive/chart-hero/datasets...
[/content/e-gmd-v1.0.0.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile director

In [ ]:
# Prepare training data (if not already processed)
# This cell converts the raw EGMD data to transformer-compatible format

import sys
sys.path.append('/content/chart-hero')

from model_training.data_preparation import data_preparation
from model_training.transformer_data import convert_legacy_data

# Process raw EGMD data
egmd_dir = "/content/drive/MyDrive/chart-hero/datasets/expanded-groove-midi"
processed_dir = "/content/drive/MyDrive/chart-hero/datasets/processed"

if not os.path.exists(processed_dir):
    print("Processing raw EGMD data...")

    # Create data preparation instance
    data_prep = data_preparation(
        directory_path=egmd_dir,
        dataset='egmd',
        sample_ratio=1.0,
        diff_threshold=1.0
    )

    # Create audio set with batching
    data_prep.create_audio_set(
        pad_before=0.1,
        pad_after=0.1,
        fix_length=10.0,  # 10 second segments
        batching=True,
        dir_path=processed_dir,
        num_batches=20
    )

    print("Data processing completed!")
else:
    print("Processed data already exists.")

!ls -la {processed_dir}

## 3. Model Training

In [ ]:
# Set up W&B logging
import wandb

# Login to W&B (you'll need to provide your API key)
wandb.login()

# Or set the API key directly
# wandb.login(key="your-wandb-api-key")

In [ ]:
# Test transformer setup
!python model_training/test_transformer_setup.py

In [ ]:
# Start training with cloud configuration
DATA_DIR = "/content/drive/MyDrive/chart-hero/datasets/processed"
AUDIO_DIR = "/content/drive/MyDrive/chart-hero/datasets/expanded-groove-midi"

!python model_training/train_transformer.py \
    --config cloud \
    --data-dir {DATA_DIR} \
    --audio-dir {AUDIO_DIR} \
    --project-name chart-hero-transformer-colab

## 4. Resume Training (if needed)

In [ ]:
# Resume from checkpoint
CHECKPOINT_PATH = "/content/drive/MyDrive/chart-hero/models/last.ckpt"

if os.path.exists(CHECKPOINT_PATH):
    !python model_training/train_transformer.py \
        --config cloud \
        --data-dir {DATA_DIR} \
        --audio-dir {AUDIO_DIR} \
        --resume {CHECKPOINT_PATH} \
        --project-name chart-hero-transformer-colab
else:
    print(f"Checkpoint not found: {CHECKPOINT_PATH}")

## 5. Model Evaluation and Export

In [ ]:
# Load and evaluate best model
import torch
from model_training.train_transformer import DrumTranscriptionModule
from model_training.transformer_config import get_config

config = get_config("cloud")
best_model_path = "/content/drive/MyDrive/chart-hero/models/best_model.ckpt"

if os.path.exists(best_model_path):
    model = DrumTranscriptionModule.load_from_checkpoint(best_model_path)
    model.eval()
    print("Model loaded successfully!")

    # Export to ONNX for deployment
    dummy_input = torch.randn(1, 1, 256, 128)
    onnx_path = "/content/drive/MyDrive/chart-hero/models/drum_transformer.onnx"

    torch.onnx.export(
        model.model,
        dummy_input,
        onnx_path,
        export_params=True,
        opset_version=11,
        do_constant_folding=True,
        input_names=['spectrogram'],
        output_names=['logits'],
        dynamic_axes={
            'spectrogram': {0: 'batch_size', 2: 'time'},
            'logits': {0: 'batch_size'}
        }
    )

    print(f"Model exported to ONNX: {onnx_path}")
else:
    print(f"Best model not found: {best_model_path}")

## 6. Cleanup

In [ ]:
# Clean up temporary files and finish W&B run
wandb.finish()

# Show final model and log locations
print("Training completed!")
print(f"Models saved to: /content/drive/MyDrive/chart-hero/models/")
print(f"Logs saved to: /content/drive/MyDrive/chart-hero/logs/")
print(f"Datasets saved to: /content/drive/MyDrive/chart-hero/datasets/")